In [2]:
from jupyter_dash import JupyterDash

from dash import Dash, dcc, html, Input, Output, dash_table, no_update  
import plotly.express as px
import json
import pandas as pd
from datetime import datetime
import numpy as np
import math
import dash_bootstrap_components as dbc
from functools import reduce
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css',
                        dbc.icons.BOOTSTRAP]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

questionIcon = '<i class="bi-question-octagon-fill" style="font-size: 1rem; color: grey;"></i>'
alertIcon = '<i class="bi-exclamation-octagon-fill" style="font-size: 1rem; color: red;"></i>'
checkIcon = '<i class="bi-check-circle-fill" style="font-size: 1rem; color: green;"></i>'

def get_svg_octagon(pct):
    if np.isnan(pct):
        return questionIcon
    else:
        return f'<i class="bi-octagon-fill" style="font-size: {pct}2rem; color: bs-info;"></i>'

with open('/home/wong/Documents/newsmf110s2.json') as f:
    data = json.load(f)

columns_wanted = ['header',
                  'dfha06ds', 
                  'dfhdsgds', 'dfhxmgds', 'dfhsmsds',
                  'dfha14ds', 'dfha20ds', 'dfha06ds', 'dfhnqgds', 'dfha17ds',
                  'dfha08ds', 'dfhd2gds', 'dfhd2rds', 'dfhmqgds']

filteredData =[]
df_dict = {}
dictfilt = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y)])

for item in data:
    newItem = dictfilt(item, columns_wanted)
    filteredData.append(newItem)
    
# Flatten data
data_norm = pd.json_normalize(filteredData)
data_norm['dateTime'] = data_norm['header.dateTime'].map(lambda x:
                              datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
    
# process dfha06ds
df_a06ds = None
if 'dfha06ds' in data_norm.keys():
    df_a06ds = pd.json_normalize([elem for elem in filteredData 
                                  if ("dfha06ds" in elem.keys() and
                                      isinstance(elem['dfha06ds'], list))], 
                             'dfha06ds',
                             meta=[['header', 'smfstprn'],
                                   ['header', 'dateTime'],
                                   ['header', 'sysId']],
                             errors='ignore',
                             record_prefix='dfha06ds.')
    df_a06ds = df_a06ds[['header.smfstprn','header.dateTime','header.sysId',
                         'dfha06ds.a06teot','dfha06ds.a06csvc',
                         'dfha06ds.a06tete','dfha06ds.a06teoe']]
    df_a06ds['dateTime'] = df_a06ds['header.dateTime'].map(lambda x:
                                  datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
    df_a06ds.drop('header.dateTime', axis=1)
    df_dict["a06ds"] = df_a06ds

# process dfhdsgds.dsgtcbm
df_dsgds = None
if 'dfhdsgds.dsgtcbm' in data_norm.keys():
    df_dsgds = pd.json_normalize([elem for elem in filteredData if "dfhdsgds" in elem.keys()], 
                             ['dfhdsgds','dsgtcbm'],
                             meta=[['header']],
                             errors='ignore',
                             record_prefix='dsgtcbm.')
    df_dsgds['header.smfstprn'] = df_dsgds.apply(lambda x: x['header']['smfstprn'], axis=1)
    df_dsgds['header.sysId'] = df_dsgds.apply(lambda x: x['header']['sysId'], axis=1)
    df_dsgds['dateTime'] = df_dsgds.apply(lambda x: x['header']['dateTime'], axis=1)
    df_dsgds['dateTime'] = df_dsgds['dateTime'].map(lambda x:
                                  datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
    df_dsgds['dsgtcbm.dsgact'] = df_dsgds['dsgtcbm.dsgact'].map(lambda x: 
                                datetime.strptime(x,"%H:%M:%S.%f") - 
                                datetime(1900,1,1))
    df_dsgds.drop(columns='header', inplace=True)

    df_dsgds = df_dsgds.loc[df_dsgds['dsgtcbm.dsgtcbnm']=='QR'][['header.smfstprn',
                                                'dateTime','header.sysId',
                                                'dsgtcbm.dsgact']]
    df_dict["dsgds"] = df_dsgds


# process dfhxmgds
df_xmgds = None
df_xmgds_1 = None
if 'dfhxmgds' in data_norm.keys():
    df_xmgds_1 = pd.json_normalize([elem for elem in filteredData 
                                    if ("dfhxmgds" in elem.keys() and
                                        isinstance(elem['dfhxmgds'], list))],
                             'dfhxmgds',
                             meta=[['header','smfstprn'],
                                   ['header', 'dateTime'],
                                   ['header', 'sysId']],
                             errors='ignore',
                             record_prefix='dfhxmgds.')
if 'dfhxmgds.xmgpat' in data_norm.keys():
    if df_xmgds_1 is None:
        df_xmgds = data_norm[['header.smfstprn','header.dateTime','header.sysId',
                              'dfhxmgds.xmgpat','dfhxmgds.xmgtamxt']].dropna()
    else:
        df_xmgds = pd.concat([df_xmgds_1[['header.smfstprn','header.dateTime','header.sysId',
                                          'dfhxmgds.xmgpat','dfhxmgds.xmgtamxt']], 
                              data_norm[['header.smfstprn','header.dateTime','header.sysId',
                                         'dfhxmgds.xmgpat','dfhxmgds.xmgtamxt']].dropna()], axis=0)
else:
    df_xmgds = df_xmgds_1

if df_xmgds is not None:
    df_xmgds['dateTime'] = df_xmgds['header.dateTime'].map(lambda x:
                                  datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
    df_xmgds = df_xmgds[['header.smfstprn','dateTime','header.sysId',
                         'dfhxmgds.xmgpat','dfhxmgds.xmgtamxt']]
        
    df_dict["xmgds"] = df_xmgds


# process dfhsmsds.smsbody
df_smsds = None
if 'dfhsmsds.smsbody' in data_norm.keys():
    df_smsds = pd.json_normalize([elem for elem in filteredData if "dfhsmsds" in elem.keys()],
                             ['dfhsmsds', 'smsbody'],
                             meta=[['header']],
                             errors='ignore',
                             record_prefix='dfhsmsds.')
    df_smsds['header.smfstprn'] = df_smsds.apply(lambda x: x['header']['smfstprn'], axis=1)
    df_smsds['header.sysId'] = df_smsds.apply(lambda x: x['header']['sysId'], axis=1)
    df_smsds['dateTime'] = df_smsds.apply(lambda x: x['header']['dateTime'], axis=1)
    df_smsds.drop(columns='header', inplace=True)
    df_smsds['dateTime'] = df_smsds['dateTime'].map(lambda x:
                                  datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
    
    df_smsds = df_smsds[['header.smfstprn','dateTime','header.sysId',
                         'dfhsmsds.smsdsaname','dfhsmsds.smssos']]
    condition1 = (df_smsds['dfhsmsds.smsdsaname'].str.startswith('E'))
    condition2 = (df_smsds['dfhsmsds.smsdsaname'].str.startswith('E','G'))
    df_smsds['dfhsmsds.smssos_EDSAs'] = df_smsds[condition1]['dfhsmsds.smssos']
    df_smsds['dfhsmsds.smssos_DSAs'] = df_smsds[~condition2]['dfhsmsds.smssos']
    df_smsds.drop('dfhsmsds.smssos', axis=1)

    df_dict["smsds"] = df_smsds

    
# process dfha17ds
df_a17ds_1 = None
df_a17ds = None
if 'dfha17ds' in data_norm.keys():
    df_a17ds_1 = pd.json_normalize([elem for elem in filteredData 
                                    if ( "dfha17ds" in elem.keys() and 
                                        isinstance(elem['dfha17ds'], list))],
                             record_path=['dfha17ds'],
                             meta=[['header','smfstprn'],
                                   ['header', 'dateTime'],
                                   ['header', 'sysId']],
                             errors='ignore',
                             record_prefix='dfha17ds.')
if 'dfha17ds.a17dshsw' in data_norm.keys():
    if df_a17ds_1 is None:
        df_a17ds = data_norm[['header.smfstprn','header.dateTime',
                              'header.sysId','dfha17ds.a17dshsw']].dropna()
    else:
        df_a17ds = pd.concat([df_a17ds_1[['header.smfstprn','header.dateTime',
                                          'header.sysId','dfha17ds.a17dshsw']],
                              data_norm[['header.smfstprn','header.dateTime',
                                         'header.sysId','dfha17ds.a17dshsw']].dropna()], axis=0)
elif df_a17ds_1 is not None:
    df_a17ds = df_a17ds_1

if df_a17ds is not None:        
    df_a17ds['dateTime'] = df_a17ds['header.dateTime'].map(lambda x:
                                  datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
    df_a17ds.drop('header.dateTime',axis=1)
    df_a17ds = df_a17ds[['header.smfstprn','dateTime',
                         'header.sysId','dfha17ds.a17dshsw']]
    df_dict["a17ds"] = df_a17ds    
    
# process dfha08ds
df_a08ds_1 = None
if 'dfha08ds' in data_norm.keys():
    df_a08ds_1 = pd.json_normalize([elem for elem in filteredData 
                                    if ( "dfha08ds" in elem.keys() and 
                                        isinstance(elem['dfha08ds'], list))],
                             record_path=['dfha08ds'],
                             meta=[['header','smfstprn'],
                                   ['header','dateTime'],
                                   ['header','sysId']],
                             errors='ignore',
                             record_prefix='dfha08ds.')
if 'dfha08ds.a08bkhsw' in data_norm.keys():
    if df_a08ds_1 is None:
        df_a08ds = data_norm[['header.smfstprn','header.dateTime',
                              'header.sysId','dfha08ds.a08bkhsw']].dropna()
    else:
        df_a08ds = pd.concat([df_a08ds_1[['header.smfstprn','header.dateTime',
                                          'header.sysId','dfha08ds.a08bkhsw']],
                              data_norm[['header.smfstprn','header.dateTime',
                                         'header.sysId','dfha08ds.a08bkhsw']].dropna()], axis=0)
else:
    df_a08ds = df_a08ds_1

if df_a08ds is not None:        
    df_a08ds['dateTime'] = df_a08ds['header.dateTime'].map(lambda x:
                                  datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
    df_a08ds = df_a08ds[['header.smfstprn','dateTime',
                         'header.sysId','dfha08ds.a08bkhsw']]
else:
    df_a08ds = pd.DataFrame(columns=['header.sysId','header.smfstprn','dateTime','dfha08ds.a08bkhsw'])
df_dict["a08ds"] = df_a08ds

# process dfha14ds
df_a14ds = None
df_a14ds_1 = None
if 'dfha14ds' in data_norm.keys():
    df_a14ds_1 = pd.json_normalize([elem for elem in filteredData 
                                    if ( "dfha14ds" in elem.keys() and 
                                        isinstance(elem['dfha14ds'], list))],
                             record_path=['dfha14ds'],
                             meta=[['header','smfstprn'],
                                   ['header','dateTime'],
                                   ['header','sysId']],
                             errors='ignore',
                             record_prefix='dfha14ds.')
if 'dfha14ds.a14estas' in data_norm.keys():
    if df_a14ds_1 is None:
        df_a14ds = data_norm[['header.smfstprn','header.dateTime',
                              'header.sysId','dfha14ds.a14estas',
                               'dfha14ds.a14estaq','dfha14ds.a14estaf',
                               'dfha14ds.a14estao','dfha14ds.a14estam']].dropna()
    else:
        df_a14ds = pd.concat([df_a14ds_1[['header.smfstprn','header.dateTime',
                                          'header.sysId','dfha14ds.a14estas',
                                          'dfha14ds.a14estaq','dfha14ds.a14estaf',
                                          'dfha14ds.a14estao','dfha14ds.a14estam']],
                              data_norm[['header.smfstprn','header.dateTime',
                                         'header.sysId','dfha14ds.a14estas',
                                          'dfha14ds.a14estaq','dfha14ds.a14estaf',
                                          'dfha14ds.a14estao','dfha14ds.a14estam']].dropna()], axis=0)
else:
    df_a14ds = df_a14ds_1

if df_a14ds is not None:        
    df_a14ds['dateTime'] = df_a14ds['header.dateTime'].map(lambda x:
                                  datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
    df_a14ds = df_a14ds[['header.smfstprn','dateTime','header.sysId',
                         'dfha14ds.a14estas','dfha14ds.a14estaq',
                         'dfha14ds.a14estaf','dfha14ds.a14estao',
                         'dfha14ds.a14estam']] 
    df_dict["a14ds"] = df_a14ds


# process dfhnqgds.nqgbody
df_nqgds = None
if 'dfhnqgds.nqgbody' in data_norm.keys():
    df_nqgds = pd.json_normalize([elem for elem in filteredData if "dfhnqgds" in elem.keys()],
                             ['dfhnqgds', 'nqgbody'],
                             meta=[['header']],
                             errors='ignore',
                             record_prefix='dfhnqgds.')
    df_nqgds['header.smfstprn'] = df_nqgds.apply(lambda x: x['header']['smfstprn'], axis=1)
    df_nqgds['header.sysId'] = df_nqgds.apply(lambda x: x['header']['sysId'], axis=1)
    df_nqgds['dateTime'] = df_nqgds.apply(lambda x: x['header']['dateTime'], axis=1)
    df_nqgds.drop(columns='header', inplace=True)
    df_nqgds['dateTime'] = df_nqgds['dateTime'].map(lambda x:
                              datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
    df_nqgds['dfhnqgds.nqgtnqwt'] = df_nqgds['dfhnqgds.nqgtnqwt'].map(lambda x: 
                                datetime.strptime(x,"%H:%M:%S.%f") - 
                                datetime(1900,1,1))
    df_nqgds['dfhnqgds.nqggnqwt'] = df_nqgds['dfhnqgds.nqggnqwt'].map(lambda x: 
                                datetime.strptime(x,"%H:%M:%S.%f") - 
                                datetime(1900,1,1))
    df_nqgds['dfhnqgds.nqgnqwt'] = df_nqgds['dfhnqgds.nqgtnqwt'] + df_nqgds['dfhnqgds.nqggnqwt']
    df_nqgds['dfhnqgds.avg_nqgnqwt'] = df_nqgds['dfhnqgds.nqgnqwt']/df_nqgds['dfhnqgds.nqgtnqsi']
    df_nqgds = df_nqgds[['header.smfstprn','dateTime','header.sysId',
                         'dfhnqgds.nqgtnqsi',
                         'dfhnqgds.nqgtnqwt',
                         'dfhnqgds.nqggnqwt',
                         'dfhnqgds.nqgnqwt',
                         'dfhnqgds.avg_nqgnqwt']]
    df_dict["nqgds"] = df_nqgds

df = reduce(lambda left, right: pd.merge(left,right, 
                            on=['header.sysId','header.smfstprn','dateTime'],
                            how='outer'), df_dict.values())

dff = df.groupby(['header.sysId','header.smfstprn']).agg({
            'dfha06ds.a06teot' : 'sum',
            'dfha06ds.a06csvc' : 'sum',
            'dfha06ds.a06tete' : 'sum',
            'dfha06ds.a06teoe' : 'sum',
            'dfhsmsds.smssos_EDSAs' : 'sum',
            'dfhsmsds.smssos_DSAs' : 'sum',
            'dfhnqgds.nqgtnqsi' : 'sum',
            'dfhnqgds.nqgtnqwt': 'sum',
            'dfhnqgds.nqggnqwt': 'sum',
            'dsgtcbm.dsgact': 'max',
            'dfhxmgds.xmgpat': 'max',
            'dfhxmgds.xmgtamxt': 'max',
            'dfha17ds.a17dshsw': 'max',
            'dfha08ds.a08bkhsw': 'max',
            'dfha14ds.a14estas': 'max',
            'dfha14ds.a14estaq': 'max',
            'dfha14ds.a14estaf': 'max',
            'dfha14ds.a14estao': 'max',
            'dfha14ds.a14estam': 'max',
            }).reset_index()

dff['dfhnqgds.nqgnqwt'] = dff[['dfhnqgds.nqgtnqwt','dfhnqgds.nqggnqwt']].sum(axis=1)
dff['dfhnqgds.avg_nqgnqwt'] = dff['dfhnqgds.nqgnqwt']/dff['dfhnqgds.nqgtnqsi']
dff['SOS on any of EDSAs']=dff['dfhsmsds.smssos_EDSAs'].apply(lambda 
                                x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))
dff['SOS on any of DSAs']=dff['dfhsmsds.smssos_DSAs'].apply(lambda 
                                x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))
dff['Term Xmit Err']=dff['dfha06ds.a06tete'].apply(lambda 
                                x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))
dff['Term Xact Err']=dff['dfha06ds.a06teoe'].apply(lambda 
                                x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))
dff['Term Storage Violation']=dff['dfha06ds.a06csvc'].apply(lambda 
                                x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))
dff['Waits for VSAM File Str']=dff['dfha17ds.a17dshsw'].apply(lambda 
                                x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))
   
dff['Waits for LSR Pool Str']=dff['dfha08ds.a08bkhsw'].apply(lambda 
                                x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))

dff['Fail Link Alloc']=dff['dfha14ds.a14estaf'].apply(lambda 
                                x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))
dff['Fail Other Reasons']=dff['dfha14ds.a14estao'].apply(lambda 
                                x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))
dff['Max O/S Alloc']=dff['dfha14ds.a14estam'].apply(lambda 
                                x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))
dff['Times at MAXTASK']=dff['dfhxmgds.xmgtamxt'].apply(lambda 
                            x:alertIcon if x > 0 else(checkIcon if x == 0 else questionIcon))    
dff['Total Enq Issued Rank']=dff['dfhnqgds.nqgtnqsi'].rank(pct=True)
dff['Total Enq Issued'] = dff['Total Enq Issued Rank'].map(lambda
                                x:get_svg_octagon(x))
dff['Peak Active User Xacts Rank']=dff['dfhxmgds.xmgpat'].rank(pct=True)
dff['Peak Active User Xacts'] = dff['Peak Active User Xacts Rank'].map(lambda
                                x:get_svg_octagon(x))
dff['Term Xacts Rank']=dff['dfha06ds.a06teot'].rank(pct=True)
dff['Term Xacts'] = dff['Term Xacts Rank'].map(lambda
                                x:get_svg_octagon(x))
dff['Total Alloc Rank']=dff['dfha14ds.a14estas'].rank(pct=True)
dff['Total Alloc'] = dff['Total Alloc Rank'].map(lambda
                                x:get_svg_octagon(x))
dff['Queue Alloc Rank']=dff['dfha14ds.a14estaq'].rank(pct=True)
dff['Queue Alloc'] = dff['Queue Alloc Rank'].map(lambda
                                x:get_svg_octagon(x))
    
dff['Highest QR TCB Rank'] = dff['dsgtcbm.dsgact'].rank(pct=True)
dff['Highest QR TCB'] = dff['Highest QR TCB Rank'].map(lambda
                                x:get_svg_octagon(x))
dff['Total Enq Wait Time Rank'] = dff['dfhnqgds.nqgnqwt'].rank(pct=True)
dff['Total Enq Wait Time'] = dff['Total Enq Wait Time Rank'].map(lambda
                                x:get_svg_octagon(x))
dff['Avg Enq Wait Time Rank'] = dff['dfhnqgds.avg_nqgnqwt'].rank(pct=True)
dff['Avg Enq Wait Time'] = dff['Avg Enq Wait Time Rank'].map(lambda
                                x:get_svg_octagon(x))
dff = dff.rename({'header.sysId': 'LPAR',
                  'header.smfstprn': 'Applid'},axis=1)

dff["id"] = dff.index

columns = {'LPAR' : 'header.sysId', 'Applid': 'header.smfstprn', 
                    'SOS on any of EDSAs': 'dfhsmsds.smssos_EDSAs', 
                    'SOS on any of DSAs': 'dfhsmsds.smssos_DSAs',
                    'Term Xmit Err': 'dfha06ds.a06tete', 
                    'Term Xact Err': 'dfha06ds.a06teoe',
                    'Term Storage Violation': 'dfha06ds.a06csvc', 
                    'Waits for VSAM File Str': 'dfha17ds.a17dshsw',
                    'Waits for LSR Pool Str': 'dfha08ds.a08bkhsw',
                    'Fail Link Alloc': 'dfha14ds.a14estaf', 
                    'Fail Other Reasons': 'dfha14ds.a14estao', 
                    'Max O/S Alloc': 'dfha14ds.a14estam',
                    'Times at MAXTASK': 'dfhxmgds.xmgtamxt', 
                    'Total Enq Issued': 'dfhnqgds.nqgtnqsi',
                    'Peak Active User Xacts': 'dfhxmgds.xmgpat', 
                    'Term Xacts': 'dfha06ds.a06teot',
                    'Total Alloc': 'dfha14ds.a14estas', 
                    'Queue Alloc': 'dfha14ds.a14estaq',
                    'Highest QR TCB': 'dsgtcbm.dsgact', 
                    'Total Enq Wait Time': 'dfhnqgds.nqgnqwt',
                    'Avg Enq Wait Time': 'dfhnqgds.avg_nqgnqwt'}

initial_active_cell = {"row": 0, "column": 0, "column_id": "Peak Active User Xacts", "row_id": 0}

app.layout = html.Div(
    [
        html.Div(
            [
                html.H3("CICS Healthiness", style={"textAlign":"center"}),
                dash_table.DataTable(
                    id="table",
                    columns=[{"name": c, "id": c, "presentation":"markdown"} for c in columns.keys()],
                    data=dff.to_dict("records"),
                    page_size=10,
                    sort_action="native",
                    active_cell=initial_active_cell,
                    markdown_options={'html': True},
                    style_cell = {

                        'minWidth': '30px','width': '30px', 'maxWidth': '60px',
                        'whiteSpace': 'normal'
                    },
                ),
            ],
            style={"margin": 50},
            # className="five columns"
        ),
        html.Div(id="output-graph", className="six columns"),
    ],
    className="row"
)


@app.callback(
      Output("output-graph", "children"),
      Input("table", "active_cell"),
  )
def cell_clicked(active_cell):
    if active_cell is None:
        return no_update
    
    row = active_cell["row_id"]

    applid = dff.at[row, "Applid"]
    
    lparid = dff.at[row, "LPAR"]
    
    col = active_cell["column_id"]

    df_s = df[
                    (df["header.sysId"]==lparid)  &
                    (df["header.smfstprn"]==applid)]
       
    y = columns[col] if col in columns.keys() else columns['Peak Active User Xacts']

    fig = px.scatter(
        df_s, x="dateTime", y=y, title=" ".join([applid, col])
    )
    fig.update_layout(title={"font_size": 20},  title_x=0.5, margin=dict(t=190, r=15, l=5, b=5))
    #fig.update_traces(line=dict(color=color[y]))

    return dcc.Graph(figure=fig)


#if __name__ == "__main__":
app.run_server(debug=True, mode='inline') #, use_reloader=False)